# Feature Extraction and Generate Vectors

In [2]:
#load lib
import os
import string
import pandas as pd
from stopwords import get_stopwords
from hazm import word_tokenize,sent_tokenize
from hazm import POSTagger,Normalizer
import re
from sklearn.decomposition import PCA
stopwords = get_stopwords('persian')

In [65]:
fa_short_words = ['و','که','را','از','به','با','در','تا','این','آن','است','بود']
fa_rabt_words = ['و','که','هم','چون','اگر','با','یا','تا','پس','چه','اما','ولی']
fa_char = []

In [70]:
lst_vectors1 = []
lst_vectors2 = []
lst_vectors3 = []
for i in range(1,41):
    text_char_count = 0
    for j in range(1,6):
        path = 'Dataset-Text/{0}/{1}-{2}.txt'.format(i,i,j)
        if (os.path.isfile(path)):
            f = open('Dataset-Text/{0}/{1}-{2}.txt'.format(i,i,j),"r",encoding="utf-8")
            text = f.read()
            
            #lst_vectors2.append(stCounting(text)) # حروف ربط
            
            #lst_vectors3.append(CountTxtTags(text)) # تگ کلمات
            
            # فراوانی توزیع حروف
            tokens = word_tokenize(text)
            filtered_text = [word for word in tokens if word not in stopwords]
            text = ' '.join(filtered_text)
            text_char_count = len(text.replace(' ','').replace('\u200c','').translate(str.maketrans('', '', string.punctuation)).replace('،','').replace('«','').replace('»',''))
            lst_vectors1.append(count_farsi_letters(text,i,text_char_count))
            

In [8]:
# Extracting the distribution of letters

farsi_letters = "ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی"

def count_farsi_letters(text,aut_name,text_char_count):
    letter_count = []
    letter_count.append(aut_name)
    for letter in farsi_letters:
        if letter in text:
            letter_count.append(int((text.count(letter)/text_char_count) * 10 ** 3) / 10 ** 3)
            #letter_count.append(int((text.count(letter)/text_char_count) * 10 ** 2) / 10 ** 2)
        else:
            letter_count.append(0)
    return letter_count
    #print(letter_count)

In [9]:
# Extracting the distribution of conjunctions - حروف ربط

word_count = []
txt_total_st_count = 0
def stCounting(txt):
    word_count = []
    txt_total_st_count = 0
    txt_token = txt.split(None)
    for wsw in fa_rabt_words:
        c = txt_token.count(wsw)
        txt_total_st_count = txt_total_st_count + c
    for wsw in fa_rabt_words:
        st = txt_token.count(wsw)
        if (st > 0):
            word_count.append(int((st / txt_total_st_count) * 10 ** 3) / 10 ** 3)
        else:
            word_count.append(0)
    return word_count

In [10]:
# Extracting the distribution of pos-tags

lst_tag_count_all =[]
normalizer = Normalizer()
model_path = 'pos_tagger.model'
tagger = POSTagger(model = model_path)

def CountTxtTags(txt):
    normalize_text = normalizer.normalize(txt)
    tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(normalize_text)]
    token_tag_list = tagger.tag_sents(tokenize_text)
    ADP=0
    NOUN=0 # اسم
    CCONJ=0 # حرف ربط
    ADJ=0 # صفت
    PUNCT=0 # علائم نگارشی
    ADV=0
    VERB=0 #فعل
    PRON=0 #ضمیر
    SCONJ=0 #
    all_tag_count = 0
    lst_tag_count =[]
    for sent in token_tag_list:
        for tag in sent:
            #print(tag[1])
            if (tag[1]=='ADP'): 
                ADP = ADP +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='NOUN'): 
                NOUN = NOUN +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='CCONJ'): 
                CCONJ = CCONJ +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='ADJ'): 
                ADJ = ADJ +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='PUNCT'): 
                PUNCT = PUNCT +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='ADV'): 
                ADV = ADV +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='VERB'): 
                VERB = VERB +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='PRON'): 
                PRON = PRON +1
                all_tag_count = all_tag_count + 1
            elif (tag[1]=='SCONJ'): 
                SCONJ = SCONJ +1
                all_tag_count = all_tag_count + 1
    ADP = (int((ADP / all_tag_count) * 10 ** 3) / 10 ** 3)
    NOUN = (int((NOUN / all_tag_count) * 10 ** 3) / 10 ** 3)
    CCONJ = (int((CCONJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    ADJ = (int((ADJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    PUNCT = (int((PUNCT / all_tag_count) * 10 ** 3) / 10 ** 3)
    ADV = (int((ADV / all_tag_count) * 10 ** 3) / 10 ** 3)
    VERB = (int((VERB / all_tag_count) * 10 ** 3) / 10 ** 3)
    PRON = (int((PRON / all_tag_count) * 10 ** 3) / 10 ** 3)
    SCONJ = (int((SCONJ / all_tag_count) * 10 ** 3) / 10 ** 3)
    lst_tag_count=[ADP,NOUN,CCONJ,ADJ,PUNCT,ADV,VERB,PRON,SCONJ]
    #lst_tag_count_all.append(lst_tag_count)
    #print(all_tag_count)
    return lst_tag_count

In [71]:
# Save vectors to excel files

df = pd.DataFrame(lst_vectors1) # convert list to dataframe

writer = pd.ExcelWriter('Vec.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='autors vector', index=False)
writer.save()
writer.close()

# normalization

In [39]:
#read ds
excel_data_df = pd.read_excel('vec-pdf-40-final-no norm.xlsx', sheet_name='autors vector')

In [40]:
df_s = excel_data_df.iloc[:,1:54].copy()

#df_s1 = excel_data_df.iloc[:,1:33].copy()
#df_s2 = excel_data_df.iloc[:,33:45].copy() 
#df_s3 = excel_data_df.iloc[:,45:54].copy() 

In [41]:
df_s

,1,2,3,4,5,6,7.1,8,9,10,...,44,45,46,47,48,49,50,51,52,53
0,0.093,0.038,0.007,0.048,0.000,0.006,0.008,0.003,0.026,0.081,...,0.129,0.058,0.287,0.055,0.035,0.169,0.046,0.275,0.036,0.033
1,0.127,0.028,0.008,0.055,0.000,0.008,0.007,0.009,0.022,0.075,...,0.120,0.110,0.265,0.100,0.057,0.109,0.058,0.212,0.044,0.041
2,0.112,0.039,0.004,0.040,0.000,0.013,0.010,0.005,0.024,0.093,...,0.086,0.093,0.303,0.078,0.037,0.102,0.042,0.214,0.056,0.071
3,0.105,0.038,0.004,0.038,0.001,0.014,0.009,0.006,0.023,0.077,...,0.090,0.101,0.293,0.083,0.037,0.116,0.041,0.212,0.045,0.067
4,0.091,0.042,0.012,0.047,0.000,0.010,0.005,0.009,0.015,0.074,...,0.072,0.116,0.278,0.086,0.054,0.099,0.049,0.215,0.050,0.049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.122,0.037,0.008,0.038,0.001,0.007,0.005,0.009,0.020,0.083,...,0.056,0.088,0.298,0.053,0.041,0.123,0.078,0.227,0.046,0.041
191,0.120,0.051,0.005,0.043,0.001,0.005,0.005,0.013,0.020,0.069,...,0.076,0.094,0.294,0.030,0.039,0.142,0.039,0.236,0.093,0.029
192,0.115,0.046,0.008,0.042,0.001,0.005,0.002,0.009,0.017,0.068,...,0.096,0.100,0.344,0.038,0.047,0.125,0.034,0.221,0.060,0.027
193,0.101,0.041,0.006,0.047,0.000,0.010,0.007,0.010,0.025,0.069,...,0.071,0.084,0.300,0.050,0.043,0.144,0.068,0.227,0.052,0.027


In [42]:
df_s_scaled = df_s.copy()

#df_s1_scaled = df_s1.copy()
#df_s2_scaled = df_s2.copy()
#df_s3_scaled = df_s3.copy()

In [43]:
# apply normalization techniques 
for column in df_s_scaled.columns: 
    df_s_scaled[column] = (df_s_scaled[column] - df_s_scaled[column].min()) / (df_s_scaled[column].max() - df_s_scaled[column].min())

#for column in df_s1_scaled.columns: 
#    df_s1_scaled[column] = (df_s1_scaled[column] - df_s1_scaled[column].min()) / (df_s1_scaled[column].max() - df_s1_scaled[column].min())
    
#for column in df_s2_scaled.columns: 
#    df_s2_scaled[column] = (df_s2_scaled[column] - df_s2_scaled[column].min()) / (df_s2_scaled[column].max() - df_s2_scaled[column].min())
    
#for column in df_s3_scaled.columns: 
#    df_s3_scaled[column] = (df_s3_scaled[column] - df_s3_scaled[column].min()) / (df_s3_scaled[column].max() - df_s3_scaled[column].min())

In [44]:
# view normalized data 
display(df_s_scaled) 

,1,2,3,4,5,6,7.1,8,9,10,...,44,45,46,47,48,49,50,51,52,53
0,0.123077,0.263158,0.294118,0.463415,0.0,0.222222,0.411765,0.166667,0.633333,0.456140,...,0.586364,0.042857,0.455056,0.300752,0.056818,0.787709,0.478873,1.000000,0.308511,0.417722
1,0.646154,0.000000,0.352941,0.634146,0.0,0.333333,0.352941,0.500000,0.500000,0.350877,...,0.545455,0.414286,0.331461,0.639098,0.306818,0.452514,0.647887,0.546763,0.393617,0.518987
2,0.415385,0.289474,0.117647,0.268293,0.0,0.611111,0.529412,0.277778,0.566667,0.666667,...,0.390909,0.292857,0.544944,0.473684,0.079545,0.413408,0.422535,0.561151,0.521277,0.898734
3,0.307692,0.263158,0.117647,0.219512,0.2,0.666667,0.470588,0.333333,0.533333,0.385965,...,0.409091,0.350000,0.488764,0.511278,0.079545,0.491620,0.408451,0.546763,0.404255,0.848101
4,0.092308,0.368421,0.588235,0.439024,0.0,0.444444,0.235294,0.500000,0.266667,0.333333,...,0.327273,0.457143,0.404494,0.533835,0.272727,0.396648,0.521127,0.568345,0.457447,0.620253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.569231,0.236842,0.352941,0.219512,0.2,0.277778,0.235294,0.500000,0.433333,0.491228,...,0.254545,0.257143,0.516854,0.285714,0.125000,0.530726,0.929577,0.654676,0.414894,0.518987
191,0.538462,0.605263,0.176471,0.341463,0.2,0.166667,0.235294,0.722222,0.433333,0.245614,...,0.345455,0.300000,0.494382,0.112782,0.102273,0.636872,0.380282,0.719424,0.914894,0.367089
192,0.461538,0.473684,0.352941,0.317073,0.2,0.166667,0.058824,0.500000,0.333333,0.228070,...,0.436364,0.342857,0.775281,0.172932,0.193182,0.541899,0.309859,0.611511,0.563830,0.341772
193,0.246154,0.342105,0.235294,0.439024,0.0,0.444444,0.352941,0.555556,0.600000,0.245614,...,0.322727,0.228571,0.528090,0.263158,0.147727,0.648045,0.788732,0.654676,0.478723,0.341772


In [48]:
# concat columns
df_final = pd.concat([df_s_scaled], axis=1)

# PCA

In [ ]:
pca = PCA(n_components=44)
pca_features = pca.fit_transform(df_s)

In [ ]:
print(sum(pca.explained_variance_ratio_))

In [ ]:
print('Shape before PCA: ', df_final.shape)
print('Shape after PCA: ', pca_features.shape)

In [ ]:
pca_features

In [ ]:
pca_df = pd.DataFrame(data=pca_features)

In [ ]:
pca_df

In [45]:
writer = pd.ExcelWriter('vec-pdf-norm-40.xlsx', engine='xlsxwriter')
df_s_scaled.to_excel(writer, sheet_name='autors vector', index=False)
writer.save()
writer.close()

C:\Users\im\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
